In [2]:
import torch
from opt import config_parser
from data import dataset_dict
import numpy as np
import json
import os

In [3]:
scene = 'chair'
root_dir = f'/mnt/hdd/mvsnerf_data/nerf_synthetic/{scene}'
file = 'transforms_val.json'

with open(os.path.join(root_dir, file), 'r') as f:
    meta = json.load(f)

In [4]:
meta

{'camera_angle_x': 0.6911112070083618,
 'frames': [{'file_path': './val/r_0',
   'rotation': 0.012566370614359171,
   'transform_matrix': [[-0.012968253344297409,
     0.44961410760879517,
     -0.8931288123130798,
     -3.6003170013427734],
    [-0.9999159574508667,
     -0.005831199698150158,
     0.011583293788135052,
     0.046693746000528336],
    [-4.656612873077393e-10,
     0.8932039141654968,
     0.44965195655822754,
     1.8126047849655151],
    [0.0, 0.0, 0.0, 1.0]]},
  {'file_path': './val/r_1',
   'rotation': 0.012566370614359171,
   'transform_matrix': [[0.13482968509197235,
     0.37570297718048096,
     -0.9168796539306641,
     -3.6960597038269043],
    [-0.990868866443634,
     0.05112272500991821,
     -0.12476181238889694,
     -0.5029308795928955],
    [3.725290298461914e-09,
     0.9253289103507996,
     0.37916529178619385,
     1.5284639596939087],
    [0.0, 0.0, 0.0, 1.0]]},
  {'file_path': './val/r_2',
   'rotation': 0.012566370614359171,
   'transform_matrix

In [82]:
img_idx = torch.load('configs/pairs.th')[f'dtu_train']

In [83]:
scene = 1

dataset = dataset_dict['dtu_ft']

cmd = f'--datadir /mnt/hdd/mvsnerf_data/dtu/scan{scene}  \
     --dataset_name dtu_ft  \
     --net_type v0 --ckpt ./ckpts/mvsnerf-v0.tar '

args = config_parser(cmd.split())

dataset_train = dataset(args, split='train')

==> image down scale: 1.0
===> training index: [25, 21, 33, 22, 14, 15, 26, 30, 31, 35, 34, 43, 46, 29, 16, 36]


In [84]:
with open('./configs/dtu_full_pairs.txt', 'w') as f:
    
    f.write('16\n')
    
    for num, i in enumerate(img_idx):
        
        f.write(f'{i}\n15 ')

        # find nearest image idx
        positions = dataset_train.poses[:,:3,3]
        dis = np.sum(np.abs(positions - dataset_train.poses[num,:3,3]), axis=-1)
        pair_dis = np.sort(dis)[1:]
        pair_idx = np.argsort(dis)[1:]
        pair_idx = [dataset_train.img_idx[item] for item in pair_idx]

        for idx, dist in zip(pair_idx, pair_dis):
            f.write(f'{idx} {dist:.3f} ')
        f.write('\n')